In [2]:
import time
from selenium import webdriver
import Queue as Q
import pandas as pd
import numpy as np

driver = webdriver.Chrome('/home/rsk/Downloads/chromedriver')

driver.get('https://en100.tribalwars.net/game.php?village=2502')

uname = driver.find_element_by_name('username')
uname.send_keys('rsk05')


pword = driver.find_element_by_name('password')
pword.send_keys('swaminathan')

login= driver.find_element_by_class_name('btn-login')
login.click()

time.sleep(3)
driver.get('https://www.tribalwars.net/en-dk/page/play/en101')

In [100]:
def getUnits():
    
    driver.get('https://en99.tribalwars.net/game.php?screen=overview')
    
    unitsData = driver.find_element_by_id("show_units")
    
    t=unitsData.text
    t = t.split("\n")
    t = t[1:-1]
    
    unitsInfo={}
    
    unitNames = {'Spear':'spear','Swordsmen':'sword','Axemen':'axe','Archers':'archer','Scouts':'spy','Light':'light','RoshanSK':'knight'}
    
    for types in t:
        types = types.split(" ")
        unitsInfo[unitNames[types[1]]] = int(types[0])
        
    return(unitsInfo)
    

In [127]:
import time

In [101]:
getUnits()

{'archer': 15,
 'axe': 132,
 'knight': 1,
 'light': 112,
 'spear': 800,
 'spy': 69,
 'sword': 941}

In [3]:
def sendAttack(coords,units):
    
    currentURL = driver.current_url
    
    if currentURL[-8:] == 'overview':
        newURL = currentURL[:-8]+'place'
        driver.get(newURL)
    else:
        driver.get('https://en101.tribalwars.net/game.php?screen=place')
    
    coordsInput = driver.find_element_by_name('input')
    coordsInput.send_keys(str(coords[0])+'|'+str(coords[1]))
    
    for unitType in units:
    
        availUnits = int(driver.find_element_by_id('units_entry_all_'+unitType).text[1:-1])
        
        #check if required units are available
        if availUnits < units[unitType]:
            return(0)
        
        unitInput = driver.find_element_by_name(unitType)
        unitInput.send_keys(units[unitType])
    
    attackButton = driver.find_element_by_name('attack')
    attackButton.click()
    
    sendButton = driver.find_element_by_name('submit')
    sendButton.click()
    
    times = driver.find_element_by_id('serverTime').text
    date = driver.find_element_by_id('serverDate').text
    timeDate = times+' '+date
    
    return(timeDate)

In [4]:
    
lcTemp3 = {'light':3,'spy':1}
lcTemp4 = {'light':3,'spy':1}

spTemp = {'spear':30,'axe':6,'spy':1}
ssTemp = {'spear':30,'sword':30,'spy':1}
ssTemp2 = {'spear':10,'sword':6,'axe':2}


In [5]:
def startFarm(fileName):

    data = pd.read_csv(fileName)

    data['lastAttacked'] = data['lastAttacked'].apply(lambda x : pd.to_datetime(x,format='%Y-%m-%d %H:%M:%S'))

    data = data.sort_values('lastAttacked')


    for i in range(len(data)):
        coords = [data.iloc[i]['x'],data.iloc[i]['y']]
        key = data.iloc[i]['key']

        temp = data.iloc[i]['template']

        if temp=='lcTemp3':
            out = sendAttack(coords,lcTemp3)
        elif temp=='lcTemp4':
            out = sendAttack(coords,lcTemp4)
        elif temp=='ssTemp2':
            out = sendAttack(coords,ssTemp2)

        if out==0:
            break
        else:
            data.loc[data.key==key,'lastAttacked'] = pd.to_datetime(out,format='%H:%M:%S %d/%m/%Y')


    data.to_csv(fileName,index=False,index_label=False)
    

In [8]:
startFarm('closeBarbsList101.csv')

In [ ]:

data = pd.read_csv('farBarbsList.csv')

data['lastAttacked'] = data['lastAttacked'].apply(lambda x : pd.to_datetime(x,format='%Y-%m-%d %H:%M:%S'))

data = data.sort_values('lastAttacked')


for i in range(len(data)):
    coords = [data.iloc[i]['x'],data.iloc[i]['y']]
    key = data.iloc[i]['key']
    
    temp = data.iloc[i]['template']
    
    if temp=='lcTemp3':
        out = sendAttack(coords,lcTemp3)
    elif temp=='lcTemp4':
        out = sendAttack(coords,lcTemp4)
        
    if out==0:
        break
    else:
        data.loc[data.key==key,'lastAttacked'] = pd.to_datetime(out,format='%H:%M:%S %d/%m/%Y')
        print(data.loc[data.key==key,'lastAttacked'])
        

data.to_csv('farBarbsList.csv',index=False,index_label=False)
        

In [109]:
farBarbsList=[[488,481],
             [488,477],
             [490,477],
             [489,480],
             [490,481],
             [490,482],
             [491,481],
             [491,483],
             [491,484],
             [485,483],
             [482,479],
             [483,475],
             [484,473],
             [488,486],
             [491,485],
             [493,484],
             [492,487],
             [491,488],
             [493,488],
             [492,489]]

for i in range(len(farBarbsList)):
    farBarbsList[i] = (0,farBarbsList[i])

In [115]:
a = np.array([x[1][0] for x in farBarbsList])
b = np.array([x[1][1] for x in farBarbsList])

df = pd.DataFrame(np.array([a,b]).T,columns = ['x','y'])

df['key'] = df.apply(lambda x : str(x['x'])+str(x['y']),axis=1)
df['points'] = df.apply( lambda x : 0, axis=1)
df['lastAttacked'] = df.apply( lambda x : pd.to_datetime('20:24:27 10/05/2018', format='%H:%M:%S %d/%m/%Y'), axis=1)
df['prevFarm'] = df.apply(lambda x: 0, axis=1)
df['prevScout'] = df.apply(lambda x: 0, axis=1)
df['template'] = df.apply(lambda x:'ssTemp2', axis=1)
df['fullReturns'] = df.apply(lambda x : 0, axis=1)
df['totalFarm'] = df.apply(lambda x : 0, axis=1)
df['totalAttacks'] = df.apply(lambda x : 0, axis=1)

df.to_csv('closeBarbsList101.csv',index=False,index_label=False)


In [116]:
len(farBarbsList)

20

In [ ]:
closeBarbsList=[(0,[610,381]),
               (0,[608,378]),
               (0,[607,381]),
               (0,[613,379]),
                (0,[611,383]),
                (0,[609,383]),
                (0,[608,375]),
                (0,[609,374]),
                (0,[608,374]),
                (0,[608,373]),
                (0,[606,374]),
                (0,[612,373]),
                (0,[614,381]),
                (0,[615,381]),
                (0,[614,382])
               ]

In [ ]:
farBarbsList=[[603,373],
             [602,373],
             [601,373],
             [600,373],
              [598,374],
              [601,377],
              [600,377],
              [603,379],
              [598,380],
              [599,381],
              [600,381],
              [598,383],
              [599,383],
              [597,384],
              [598,384],
              [597,385],
              [599,386],
              [600,386],
              [602,386],
              [598,386],
              [607,386],
              [606,387],
              [604,386],
              [604,388],
              [605,389],
              [604,389],
              [607,391]]

for i in range(len(farBarbsList)):
    farBarbsList[i] = (0,farBarbsList[i])
        